In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog
import cv2
from PIL import Image, ImageTk
from keras.models import load_model
import numpy as np

In [ ]:


class FruitSpoilageApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Fruit Spoilage Detection")
        self.root.geometry("1000x800")  # Set the initial size of the GUI
        
        # Add a label for the background image
        self.background_image = tk.PhotoImage(file="bg_4.png")
        background_label = tk.Label(root, image=self.background_image)
        background_label.place(relwidth=1, relheight=1)

        # Load the pre-trained model
        self.model_first = load_model('best_model_with_humans_and_drawing_added.keras')  
        self.model_second = load_model('best_model_3_very_good_prediction_trained_on_newest_dataset_3_epochs_25_10.keras')

        # Set the style for rounded buttons
        style = ttk.Style()
        style.configure("TButton", padding=6, relief="flat", background="#88d7f4", borderwidth=0)
        style.map("TButton", foreground=[("active", "white")])

        # Create a label for the title
        title_label = tk.Label(root, text="Fruit Spoilage Detection", font=("Helvetica", 24, "bold"), fg="black")
        title_label.pack(pady=20)

        # Create a label for the camera feed with a black border
        self.camera_label = tk.Label(root, bd=2, relief="solid", highlightbackground="black")
        self.camera_label.pack()

        # Create a label for displaying the result
        self.result_label = tk.Label(root, text="", font=("Helvetica", 16), fg="red", background="#88d7f4")
        self.result_label.pack(pady=12)

        # Create a button for capturing an image
        self.capture_button = ttk.Button(root, text="Capture Image", command=self.capture_image)
        self.capture_button.pack(pady=12, ipadx=15, ipady=8)

        # Create a button for recapturing the image
        self.recapture_button = ttk.Button(root, text="Recapture", command=self.recapture_image)
        self.recapture_button.pack(pady=12, ipadx=15, ipady=8)

        # Create a button for processing the captured image
        self.process_button = ttk.Button(root, text="Process Image", command=self.process_image)
        self.process_button.pack(pady=12, ipadx=15, ipady=8)

        # Create a button for loading an image
        self.load_image_button = ttk.Button(root, text="Load Image", command=self.load_image)
        self.load_image_button.pack(pady=12, ipadx=15, ipady=8)

        # Set the initial state of recapture and process buttons
        self.recapture_button.config(state=tk.DISABLED)
        self.process_button.config(state=tk.DISABLED)

        # Initialize camera flag
        self.camera_running = False

        # Open the camera feed when the app starts
        self.show_camera_feed()

        # Bind the function to close the camera when the GUI is destroyed
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def on_close(self):
        # Release the camera
        self.camera_running = False
        self.root.destroy()

    def show_camera_feed(self):
        # Open the camera feed and display it in the window
        cap = cv2.VideoCapture(2)  # 0 for default camera and can be changed
        self.camera_running = True
        self.update_camera_feed(cap)

    def update_camera_feed(self, cap):
        if self.camera_running:
            _, frame = cap.read()

            # Resize the image to match the model input shape
            frame = cv2.resize(frame, (224, 224))

            # Convert OpenCV image to Tkinter-compatible format
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(img)

            # Update the camera label with the new image and black border
            self.camera_label.config(image=img, bd=2, relief="solid", highlightbackground="black")
            self.camera_label.image = img

            # Schedule the function to be called again after 10 milliseconds (can be adjusted as needed)
            self.root.after(10, self.update_camera_feed, cap)

    def capture_image(self):
        # Stop the camera feed
        self.camera_running = False
        self.camera_label.config(image=None)

        # Capture a single frame
        cap = cv2.VideoCapture(2)  # 0 for default camera
        _, frame = cap.read()
        cap.release()

        # Resize the image to match the model input shape
        frame = cv2.resize(frame, (224, 224))

        # Convert OpenCV image to Tkinter-compatible format
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(img)

        # Update the camera label with the captured image and black border
        self.camera_label.config(image=img, bd=2, relief="solid", highlightbackground="black")
        self.camera_label.image = img

        # Save the captured image to a variable for later use
        self.captured_image = frame

        # Enable the recapture and process buttons
        self.recapture_button.config(state=tk.NORMAL)
        self.process_button.config(state=tk.NORMAL)

        # Clear previous result
        self.result_label.config(text="")

    def recapture_image(self):
        # Show the camera feed again
        self.camera_running = True
        self.show_camera_feed()

    def process_image(self):
    # Preprocess the captured image for model prediction
        processed_img = self.preprocess_image(self.captured_image)

    # Predict the class probabilities using the loaded model
        prediction_first_model = self.model_first.predict(processed_img)
        
        max_prob_index = np.argmax(prediction_first_model[0])
        
        if max_prob_index == 0 or max_prob_index == 5:
            result_message = "Not a fruit"
        
        else: 
            prediction_second_model = self.model_second.predict(processed_img)
            
            class_labels = ['freshapple', 'freshbanana', 'freshgrape', 'freshorange', 'rottenapple', 'rottenbanana', 'rottengrape', 'rottenorange']
            
            predicted_class_index = np.argmax(prediction_second_model[0])
            predicted_class_label = class_labels[predicted_class_index]
            result_message = f"Predicted Fruit Class: {predicted_class_label}"
        
        self.result_label.config(text=result_message)


    def preprocess_image(self, img):
        # Necessary preprocessing can be added here...
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0)
        img = img.astype('float32') / 255.0  # Normalize pixel values
        return img

    def load_image(self):
        # Stop the camera feed
        self.camera_running = False

        # Allow the user to select a file
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])

        if file_path:
            # Read the image from the selected file
            loaded_image = cv2.imread(file_path)

            # Resize the loaded image to match the model input shape
            loaded_image = cv2.resize(loaded_image, (280, 280))

            # Convert OpenCV image to Tkinter-compatible format
            img = cv2.cvtColor(loaded_image, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(img)

            # Update the camera label with the loaded image and black border
            self.camera_label.config(image=img, bd=2, relief="solid", highlightbackground="black")
            self.camera_label.image = img

            # Save the loaded image to a variable for later use
            self.captured_image = loaded_image

            # Enable the recapture and process buttons
            self.recapture_button.config(state=tk.NORMAL)
            self.process_button.config(state=tk.NORMAL)

            # Clear previous result
            self.result_label.config(text="")




In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = FruitSpoilageApp(root)
    root.mainloop()